In [27]:
import os
import json
import pandas as pd
import collections
import sentencepiece as spm
import io
from nltk.tokenize import sent_tokenize


In [4]:
#load our 10k data into a dataframe
papers = []
for root, dirs, files in os.walk("./data/mini_10k"):
    for f in files:
        fn = root+"/"+f
        with open(fn) as jsonfile:
            d = json.load(jsonfile)
        papers.append(d)

df = pd.DataFrame(papers)
        

In [5]:
df

,id,guidislink,updated,updated_parsed,published,published_parsed,title,title_detail,summary,summary_detail,...,arxiv_comment,links,arxiv_primary_category,tags,pdf_url,affiliation,arxiv_url,journal_reference,doi,fulltext
0,http://arxiv.org/abs/math/0407145v2,True,2004-07-08T21:21:58Z,"[2004, 7, 8, 21, 21, 58, 3, 190, 0]",2004-07-08T16:10:34Z,"[2004, 7, 8, 16, 10, 34, 3, 190, 0]",Compact packings of the plane with two sizes o...,"{'type': 'text/plain', 'language': None, 'base...",We consider packings of the plane using discs ...,"{'type': 'text/plain', 'language': None, 'base...",...,"Latex, 20 pages, 15 postscript figures Only ch...",[{'href': 'http://arxiv.org/abs/math/0407145v2...,"{'term': 'math.MG', 'scheme': 'http://arxiv.or...","[{'term': 'math.MG', 'scheme': 'http://arxiv.o...",http://arxiv.org/pdf/math/0407145v2,None,http://arxiv.org/abs/math/0407145v2,"Discrete and Computational Geometry 35 (2006),...",None,arXiv:math/0407145v2 [math.MG] 8 Jul 2004\n\nC...
1,http://arxiv.org/abs/1007.0906v1,True,2010-07-06T14:35:59Z,"[2010, 7, 6, 14, 35, 59, 1, 187, 0]",2010-07-06T14:35:59Z,"[2010, 7, 6, 14, 35, 59, 1, 187, 0]",Matrix Algebras and Semidefinite Programming T...,"{'type': 'text/plain', 'language': None, 'base...",This PhD thesis is concerned with SDP bounds f...,"{'type': 'text/plain', 'language': None, 'base...",...,"PhD Thesis (2005), about 100 pages","[{'href': 'http://arxiv.org/abs/1007.0906v1', ...","{'term': 'math.CO', 'scheme': 'http://arxiv.or...","[{'term': 'math.CO', 'scheme': 'http://arxiv.o...",http://arxiv.org/pdf/1007.0906v1,None,http://arxiv.org/abs/1007.0906v1,None,None,arXiv:1007.0906v1 [math.CO] 6 Jul 2010\n\nMatr...
2,http://arxiv.org/abs/physics/0208084v1,True,2002-08-23T16:24:36Z,"[2002, 8, 23, 16, 24, 36, 4, 235, 0]",2002-08-23T16:24:36Z,"[2002, 8, 23, 16, 24, 36, 4, 235, 0]",Resonant Absorption between Moving Atoms due t...,"{'type': 'text/plain', 'language': None, 'base...",By taking both the Doppler frequency shift for...,"{'type': 'text/plain', 'language': None, 'base...",...,None,[{'href': 'http://arxiv.org/abs/physics/020808...,"{'term': 'physics.gen-ph', 'scheme': 'http://a...","[{'term': 'physics.gen-ph', 'scheme': 'http://...",http://arxiv.org/pdf/physics/0208084v1,None,http://arxiv.org/abs/physics/0208084v1,None,None,arXiv:physics/0208084v1 [physics.gen-ph] 23 Au...
3,http://arxiv.org/abs/1112.2113v1,True,2011-12-09T15:01:25Z,"[2011, 12, 9, 15, 1, 25, 4, 343, 0]",2011-12-09T15:01:25Z,"[2011, 12, 9, 15, 1, 25, 4, 343, 0]",Incremental Slow Feature Analysis: Adaptive an...,"{'type': 'text/plain', 'language': None, 'base...",Slow Feature Analysis (SFA) extracts features ...,"{'type': 'text/plain', 'language': None, 'base...",...,None,"[{'href': 'http://arxiv.org/abs/1112.2113v1', ...","{'term': 'cs.AI', 'scheme': 'http://arxiv.org/...","[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",http://arxiv.org/pdf/1112.2113v1,None,http://arxiv.org/abs/1112.2113v1,"Neural Computation, 2012, Vol. 24, No. 11, Pag...",None,Incremental Slow Feature Analysis: Adaptive an...
4,http://arxiv.org/abs/1106.0641v1,True,2011-06-02T14:14:55Z,"[2011, 6, 2, 14, 14, 55, 3, 153, 0]",2011-06-02T14:14:55Z,"[2011, 6, 2, 14, 14, 55, 3, 153, 0]",Interval Semirings,"{'type': 'text/plain', 'language': None, 'base...",This book has seven chapters. In chapter one w...,"{'type': 'text/plain', 'language': None, 'base...",...,155 pages; Published by Kappa & Omega in 2011,"[{'href': 'http://arxiv.org/abs/1106.0641v1', ...","{'term': 'math.GM', 'scheme': 'http://arxiv.or...","[{'term': 'math.GM', 'scheme': 'http://arxiv.o...",http://arxiv.org/pdf/1106.0641v1,None,http://arxiv.org/abs/1106.0641v1,None,None,INTERVAL SEMIRINGS\n\nW. B. Vasantha Kandasamy...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,http://arxiv.org/abs/1010.3242v1,True,2010-10-15T18:42:39Z,"[2010, 10, 15, 18, 42, 39, 4, 288, 0]",2010-10-15T18:42:39Z,"[2010, 10, 15, 18, 42, 39, 4, 288, 0]",5-qubit quantum error correct

In [10]:
#analyze vocab of 10k
vocab = collections.Counter()
for i in range(len(df)):
    fulltext = df.fulltext[i]
    
    words = fulltext.lower().split()
    for w in words:
        if w not in vocab:
            vocab[w] = 0
        vocab[w] +=1
    

In [23]:
len(vocab)

with open('data/word_freq_list.tsv', 'w') as f:
    for k, v in vocab.items():
        f.write('%s\t%d\n' % (k, v))

In [28]:
#create a sentencepiece tokenizer model
spm.SentencePieceTrainer.train('--input=data/word_freq_list.tsv --input_format=tsv --model_prefix=m --vocab_size=32000')


True

In [32]:
sp = spm.SentencePieceProcessor()
sp.load('/data/m.model')
x = sp.encode_as_ids('this is a test.')
print(x)
print(sp.decode_ids(x))


[52, 16, 3, 7, 901, 4]
this is a test.


In [36]:
summary = df.summary[0].lower()
print(summary)
print(sp.encode_as_pieces(summary))
ids = sp.encode_as_ids(summary)
print(sp.decode_ids(ids))

we consider packings of the plane using discs of radius 1 and r. a packing is
compact if every disc d is tangent to a sequence of discs d_1, d_2, ..., d_n
such that d_i is tangent to d_{i+1}. we prove that there are only nine values
of r with r<1 for which such packings are possible. for each of the nine values
we describe the possible compact packings.
['▁we', '▁consider', '▁packings', '▁of', '▁the', '▁plane', '▁using', '▁discs', '▁of', '▁radius', '▁1', '▁and', '▁', 'r', '.', '▁', 'a', '▁packing', '▁is', '▁compact', '▁if', '▁every', '▁disc', '▁', 'd', '▁is', '▁tangent', '▁to', '▁', 'a', '▁sequence', '▁of', '▁discs', '▁', 'd', '_', '1', ',', '▁', 'd', '_', '2', ',', '▁...,', '▁', 'd', '_', 'n', '▁such', '▁that', '▁', 'd', '_', 'i', '▁is', '▁tangent', '▁to', '▁', 'd', '_', '{', 'i', '+1}', '.', '▁we', '▁prove', '▁that', '▁there', '▁are', '▁only', '▁nine', '▁values', '▁of', '▁', 'r', '▁with', '▁', 'r', '<', '1', '▁for', '▁which', '▁such', '▁packings', '▁are', '▁possible', '.', '▁for', '▁